In [1]:
from time import sleep
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
import re

In [ ]:
##create dataframe
df = pd.DataFrame(columns=['url', 'keyword'])
keywords = ["electric", "growth", "butter", "kirkland"]

for word in keywords:


    driver = webdriver.Chrome()

    driver.get('https://mastodon.world/explore')
    sleep(2)

    search = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div[1]/div[2]/div[2]/div/div[1]/form/input")
    search.send_keys(word, Keys.ENTER)
    sleep(2)

    hastag_button = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div[1]/div[2]/div[2]/div/div[2]/button[3]")
    hastag_button.click()
    sleep(2)

    hastag_link = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div[1]/div[2]/div[2]/div/div[3]/div/div/article[1]/div/div[1]/a")
    hastag_link.click()
    sleep(2)

    article_element = driver.find_elements(By.XPATH, "//article/div/div")

    count = 0
    index = 0
    href_links = []
    url_pattern = r"https://\S+/@\S+@\S+/\S+"
    while count < 10 and index < len(article_element):
        all_links = driver.find_elements(By.XPATH, "//a[@href]")
        for link in all_links:
            try:
                href = link.get_attribute("href")
            except StaleElementReferenceException:
                driver.execute_script("window.scrollBy(0, 500);") 
            url = re.findall(url_pattern, href)
            if url:
                if href not in href_links:
                    href_links.append(href)
                    data = {'url': href, 'keyword': word}
                    df.loc[len(df)] = data
                    count += 1
        driver.execute_script("arguments[0].scrollIntoView();", article_element[index])
        index += 1
        sleep(2)
    print(href_links)
    driver.quit()

print(df)

['https://mastodon.world/@H2ORadio@journa.host/115522937849578698', 'https://mastodon.world/@ElonMusk@activitypub.awakari.com/115522306717782749', 'https://mastodon.world/@evshift@channels.im/115521183338373118', 'https://mastodon.world/@evshift@channels.im/115521033839973320', 'https://mastodon.world/@evshift@channels.im/115520734917691396', 'https://mastodon.world/@evshift@channels.im/115520593313591567', 'https://mastodon.world/@cnbusinessforum@mstdn.business/115520451690883910', 'https://mastodon.world/@evshift@channels.im/115520428436521852', 'https://mastodon.world/@evshift@channels.im/115520145154411160', 'https://mastodon.world/@evshift@channels.im/115519342859889349']
['https://mastodon.world/@2rZiKKbOU3nTafniR2qMMSE0gwZ@activitypub.awakari.com/115523218696064004', 'https://mastodon.world/@politico_eu_bot@social.espeweb.net/115523192907784937', 'https://mastodon.world/@smohindroo1@vivaldi.net/115522974053480119', 'https://mastodon.world/@Itchigo@macrofurs.social/11552245914918

In [3]:
df.to_csv("mastodon_links.csv")